## Titanic with feature engineering. 

In [ ]:
# Imports
# pandas
import pandas as pd
from pandas import Series,DataFrame

# numpy, matplotlib, seaborn
import numpy as np
%matplotlib inline

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import GridSearchCV

Get titanic & test csv files as a DataFrame

In [ ]:
X_train = pd.read_csv("./train.csv")
X_test_orig = pd.read_csv("./test.csv")

Build the training set

In [ ]:
y_train = X_train.pop("Survived")

Combine train and test for feature engineering

In [ ]:
data = X_train.append(X_test_orig, ignore_index=True)

In [ ]:
data.head()

To Do: How many feature column? Hing: use info()

To Do: How many missing values. Hint: use isnull() and sum()

Find more useful information to learn: get the titles of each passenger 

In [ ]:
data['Title'] = data.Name.apply(lambda name: name.split(',')[1].split('.')[0].strip())
data.head()

To Do: Count diferent values of a feature. Hint: use value_counts()

Create bins for the title values

In [ ]:
bin_titles = {
    "Capt":       "Officer",
    "Col":        "Officer",
    "Major":      "Officer",
    "Jonkheer":   "Royalty",
    "Don":        "Royalty",
    "Sir" :       "Royalty",
    "Dr":         "Officer",
    "Rev":        "Officer",
    "the Countess":"Royalty",
    "Dona":       "Royalty",
    "Mme":        "Mrs",
    "Mlle":       "Miss",
    "Ms":         "Mrs",
    "Mr" :        "Mr",
    "Mrs" :       "Mrs",
    "Miss" :      "Miss",
    "Master" :    "Master",
    "Lady" :      "Royalty"
}

map the titles to title bins

In [ ]:
data.Title = data.Title.map(bin_titles)
data.Title.value_counts()

Filling Missing Values for Age feature

In [ ]:
# group by Sex, Pclass, and Title
grouped = data.groupby(['Sex','Pclass', 'Title'])

view the median Age by the grouped features

In [ ]:
grouped.Age.median()

In [ ]:
data.Age = grouped.Age.apply(lambda x: x.fillna(x.median()))

Fill Embarked with the most frequent value

In [ ]:
most_embarked = data.Embarked.value_counts().index[0]
data["Embarked"].fillna(most_embarked, inplace=True)

Fill Fare with the mean value of all fare

In [ ]:
data["Fare"].fillna(data.Fare.mean(), inplace=True)


To Do: Check null value again (and the Cabin feature will be dropped)

Apply 1-hot encoding for categorical feature Sex

In [ ]:
data = pd.get_dummies(data, columns=['Sex'], prefix = ['Sex'])

To Do: apply 1-hot encoding for the other categorical features

In [ ]:
data

Add new synthetic feature of Family Size

In [ ]:
data['Family_Size'] = data['Parch'] + data['SibSp'] + 1

Add new has_cabin feature (yes or no) 

In [ ]:
data['Has_Cabin'] = ~data.Cabin.isnull()

Binning numerical columns

In [ ]:
data['CatAge'] = pd.qcut(data.Age, q=4, labels=False )

Drop un-used features

In [ ]:
data = data.drop(['Parch','SibSp','PassengerId','Name','Ticket','Cabin', 'Age'], axis=1)

In [ ]:
data

Rebuild the training set and test set

In [ ]:
X_train = data.iloc[:891]
X_test = data.iloc[891:]

## Build Classification Model 

### Use a simple Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

split out our own test dataset from the training set.

In [ ]:
X_train_small = data.iloc[:800]
X_test_small = data.iloc[800:891]
y_train_small = y_train.iloc[:800]
y_test_small = y_train.iloc[800:891]

In [ ]:
dt1 = DecisionTreeClassifier(random_state=42)
dt1.fit(X_train_small, y_train_small)

In [ ]:
cross_val_score(dt1, X_train_small, y_train_small, cv=3, scoring="accuracy")

In [ ]:
from sklearn.metrics import roc_curve, auc
y_pred_small = dt1.predict(X_test_small)
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test_small, y_pred_small)
roc_auc = auc(false_positive_rate, true_positive_rate)
roc_auc

Fine tune hyper-parameters with GridSearchCV

In [ ]:
dt_params = [
   { 'max_depth': [1, 2, 4, 8, 16, 32, 64], 
     'min_samples_leaf' : [1, 2, 3, 4, 5, 6],
   },
]

In [ ]:
dt_cv = GridSearchCV(estimator=dt1, param_grid=dt_params, cv=4)
dt_cv.fit(X_train_small, y_train_small)

In [ ]:
cross_val_score(dt_cv.best_estimator_, X_train_small, y_train_small, cv=3, scoring="accuracy")

In [ ]:
print("Optimal params: {}".format(dt_cv.best_estimator_))

In [ ]:
from sklearn.metrics import roc_curve, auc
y_pred_small = dt_cv.best_estimator_.predict(X_test_small)
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test_small, y_pred_small)
roc_auc = auc(false_positive_rate, true_positive_rate)
roc_auc

To Do: Experiment with scaled data. Scale the test data and use decision tree classifier to see if the results in terms of AUC improve. 

To Do: Continue with the last exercise and use GridSearchCV to find the best parameter for the desicision tree classifier

### Use a random forest classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rnd_clf = RandomForestClassifier(random_state=42, n_estimators=100)
rnd_clf.fit(X_train_small, y_train_small)

In [ ]:
y_pred_small = rnd_clf.predict(X_test_small)
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test_small, y_pred_small)
roc_auc = auc(false_positive_rate, true_positive_rate)
roc_auc

Prepare prediction results for submission to Kaggle

In [ ]:
y_pred = rnd_clf.predict(X_test)

submission = pd.DataFrame({
        "PassengerId": X_test_orig["PassengerId"],
        "Survived": y_pred
    })
submission.to_csv('titanic.csv', index=False)